In [ ]:
%%time
import json
import os
import warnings

import pandas as pd
import pulumi
from datarobot_predict.deployment import predict

import datarobot as dr
import pulumi_datarobot as datarobot
from pulumi import automation as auto

if not os.getenv("DATAROBOT_NOTEBOOK_IMAGE"):
    print("not running in DataRobot Notebook")
    from dotenv import load_dotenv
    load_dotenv("../.env", override=True)
else:
    print("running in DataRobot Notebook")
    os.environ["PULUMI_CONFIG_PASSPHRASE"] = "dr"
warnings.filterwarnings("ignore")

client = dr.Client()

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.precision", 8)

In [ ]:
model_id = "67bb0f8070b10e0e4071f347"
usecase_id = os.getenv("DATAROBOT_DEFAULT_USE_CASE")
stack_name = "mlops-bleedout"
project_name = "dr-workshop"

In [ ]:
def stack_up(project_name: str, stack_name: str, program: callable) -> auto.Stack:
    # create (or select if one already exists) a stack that uses our inline program
    stack = auto.create_or_select_stack(
        stack_name=stack_name, project_name=project_name, program=program
    )

    stack.refresh(on_output=print)

    stack.up(on_output=print)
    return stack


def destroy_project(stack: auto.Stack):
    """Destroy pulumi project"""
    stack_name = stack.name
    stack.destroy(on_output=print)

    stack.workspace.remove_stack(stack_name)
    print(f"stack {stack_name} in project removed")


def make_deployment():
    """Deploy a trained model onto DataRobot."""
    user_name = client.get("account/info").json()["email"].split("@")[0]
    registered_model = datarobot.RegisteredModelFromLeaderboard(
        resource_name=f"[{user_name}]-registered-model",
        model_id=model_id,
        name=f"[{user_name}]-registered-model",
        use_case_ids=[usecase_id],
    )
    registered_model_id = registered_model.id
    registered_model_version_id = registered_model.version_id
    prediction_environment = datarobot.PredictionEnvironment(
        resource_name=f"[{user_name}]-prediction-environment",
        name=f"[{user_name}]-prediction-environment",
        batch_jobs_max_concurrent=20,
        platform="datarobotServerless",
        supported_model_formats=[
            "datarobot",
            # "customModel",
        ],
    )
    prediction_environment_id = prediction_environment.id
    deployment = datarobot.Deployment(
        resource_name=f"[{user_name}]-deployment",
        label=f"[{user_name}]-deployment",
        registered_model_version_id=registered_model_version_id,
        prediction_environment_id=prediction_environment_id,
        drift_tracking_settings={
            "feature_drift_enabled": True,
            "target_drift_enabled": True,
        },
        association_id_settings={
            "auto_generate_id": False,
            "column_names": ["association_id"],
            "required_in_prediction_requests": True,
        },
        health_settings={
            "data_drift": {
                "time_interval": "P180D",
            },
        },
        challenger_models_settings={
            "enabled": True,
        },
        predictions_by_forecast_date_settings={
            "enabled": True,
            "column_name": "date",
            "datetime_format": "%Y-%m-%d",  # 2022-01-01
        },
        predictions_data_collection_settings={
            "enabled": True,
        },
        batch_monitoring_settings={
            "enabled": False,
        },
        segment_analysis_settings={
            "enabled": True,
            "attributes": [],
        },
        use_case_ids=[usecase_id],
    )
    pulumi.export("registered_model_id", registered_model_id)
    pulumi.export("registered_model_version_id", registered_model_version_id)
    pulumi.export("prediction_environment_id", prediction_environment_id)
    pulumi.export("deployment_id", deployment.id)

In [ ]:
stack = stack_up(project_name, stack_name, program=make_deployment)

In [ ]:
result = stack.outputs()
deployment_id = result["deployment_id"].value
deployment = dr.Deployment.get(deployment_id)

### 予測

In [ ]:
df_test = pd.read_csv("../data/mlops_bleed_out_test.csv")
display(df_test.head())
df_result, _ = predict(deployment, df_test)
display(df_result.head())

### 実績値をアップロード

In [ ]:
dr.Client()
df_actuals = pd.read_csv("../data/mlops_bleed_out_actuals.csv")
deployment.submit_actuals(df_actuals)
# json_actual = df_actuals.rename(
#     columns={"association_id": "associationId", "actual_value": "actualValue"}
# ).to_dict(orient="records")
# response = client.post(
#     f"deployments/{deployment_id}/actuals/fromJSON/", json={"data": json_actual}
# )
# print(response.status_code)

### デプロイなどリソースの削除

In [ ]:
# destroy_project(stack)